In [5]:
from plyfile import PlyData
import numpy as np
import argparse
from io import BytesIO

In [6]:
def process_ply_to_splat(ply_file_path):
    plydata = PlyData.read(ply_file_path)
    vert = plydata["vertex"]
    sorted_indices = np.argsort(
        -np.exp(vert["scale_0"] + vert["scale_1"] + vert["scale_2"])
        / (1 + np.exp(-vert["opacity"]))
    )
    buffer = BytesIO()
    for idx in sorted_indices:
        v = plydata["vertex"][idx]
        position = np.array([v["x"], v["y"], v["z"]], dtype=np.float32)
        scales = np.exp(
            np.array(
                [v["scale_0"], v["scale_1"], v["scale_2"]],
                dtype=np.float32,
            )
        )
        rot = np.array(
            [v["rot_0"], v["rot_1"], v["rot_2"], v["rot_3"]],
            dtype=np.float32,
        )
        SH_C0 = 0.28209479177387814
        color = np.array(
            [
                0.5 + SH_C0 * v["f_dc_0"],
                0.5 + SH_C0 * v["f_dc_1"],
                0.5 + SH_C0 * v["f_dc_2"],
                1 / (1 + np.exp(-v["opacity"])),
            ]
        )
        buffer.write(position.tobytes())
        buffer.write(scales.tobytes())
        buffer.write((color * 255).clip(0, 255).astype(np.uint8).tobytes())
        buffer.write(
            ((rot / np.linalg.norm(rot)) * 128 + 128)
            .clip(0, 255)
            .astype(np.uint8)
            .tobytes()
        )

    return buffer.getvalue()

def save_splat_file(splat_data, output_path):
    with open(output_path, "wb") as f:
        f.write(splat_data)

In [15]:
input_files = ["/Users/ewojcik/Code/web/3d_platform/python/3d_models/input_ply/gradio.ply"]
output_file = "/Users/ewojcik/Code/web/3d_platform/python/3d_models/output_splat/Tree.splat"

for input_file in input_files:
    print(f"Processing {input_file}...")
    splat_data = process_ply_to_splat(input_file)
    output_file = (
        output_file if len(input_files) == 1 else input_file + ".splat"
    )
    save_splat_file(splat_data, output_file)
    print(f"Saved {output_file}")

Processing /Users/ewojcik/Code/web/3d_platform/python/3d_models/input_ply/gradio.ply...
Saved /Users/ewojcik/Code/web/3d_platform/python/3d_models/output_splat/Tree.splat
